### Initialisation notebook no ROOT

Collecting a bunch of useful functions and stuff for various levels of analysis

Some notebooks with useful stuff:
 * https://github.com/XENON1T/SR1Results/blob/master/Calibration/inputs_for_bbf/reduce_data.py#L162
 * 

In [1]:
#### General Python packages ####
#################################
from scipy import stats
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from matplotlib.legend_handler import HandlerPatch              # To draw circles in legends :D
import matplotlib.patches as mpatches                           # To draw circles in legends :D
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d

import sys
sys.path.append('/home/elykov/scratch_elykov2/Analysis/Wall_Leakage_Analysis/')
sys.path.append('/home/elykov/.local/lib/python3.5/site-packages/')  # for installed tabulate


from IPython.display import HTML, display
import tabulate            # table display in cell

import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm
from time import strftime
from pandas import read_hdf

%matplotlib inline                                             
# Set plots to be inline the notebook


#### Some pyROOT and that can be seful for more complicated analysis ####
#########################################################################
# import ROOT
# from ROOT import RooArgSet, RooDataSet, RooNDKeysPdf, RooRealVar,\
# RooAbsRealLValue, RooKeysPdf, RooPolynomial, RooArgList, RooBinning, RooAbsData, RooFit
#from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F, TMath, TF1, TH1

# import rootnotes                       # Need this to make plots in jupyter notebook, taken from: https://gist.github.com/mazurov/6194738
                                       # http://mazurov.github.io/webfest2013/

# from root_pandas import read_root      # Convert and read the obtained ROOT TTree to a pandas dataframe using the following function:
                                       # https://github.com/scikit-hep/root_pandas
# from root_numpy import array2tree, tree2array, fill_hist, rec2array, hist2array, array2hist


#### HAX and PAX related fucntions ####
#######################################
import hax
hax_version = hax.__version__
from hax import cuts
from pax import units, configuration, datastructure

In [2]:
# Send a message that we have finished the initialization
print('Initialization done, Notebook was last run on:', strftime("%d/%m/%Y"))
print('hax version %s' %hax_version)

Initialization done, Notebook was last run on: 28/02/2018
hax version 2.4.0


A class to draw pretty circles:

In [3]:
# A class to plot circles in the legend, was shamelessly taken from some internet website
class HandlerEllipse(HandlerPatch):
    def create_artists(self, legend, orig_handle,
                       xdescent, ydescent, width, height, fontsize, trans):
        center = 0.5 * width - 0.5 * xdescent, 0.5 * height - 0.5 * ydescent
        p = mpatches.Ellipse(xy=center, width=width + xdescent,
                             height=height + ydescent)
        self.update_prop(p, orig_handle, legend)
        p.set_transform(trans)
        return [p]

In [4]:
# Importing some useful TPC parameters
pax_config = configuration.load_configuration('XENON1T')
tpc_height = pax_config['DEFAULT']['tpc_length']
tpc_radius = pax_config['DEFAULT']['tpc_radius']
      

In [5]:
# Funcitons taken from here:
# https://github.com/XENON1T/SR1Results/blob/master/Background/WallBackground/WallBackgroundModel/wall_background_model_roi_tail_fitting.ipynb
def sigma_exp(logs2):
    y = 0.95 * logs2 - 1.47
    sigma = 1 / y
    return sigma

def plot_curve(n, c, st):
    # plot for the curve
    x = np.linspace(2, np.log10(2000), 100)
    plt.plot(47.9 - n * sigma_exp(x), x, color = c, lw=2, ls=st, label='%d $\sigma$' %n)
    return None

# Band limits taken from here:
# https://xe1t-wiki.lngs.infn.it/doku.php?id=xenon:kazama:anomalous_bg_sr1#appendix_2er_nr-band_lines_estimated_with_pax_v680_3d_fdc_and_1300_kg_fvdef ng_nr_mean(cs1):
def ng_nr_mean(cs1):
    y = 0.170987 * np.exp(-cs1 / 40.0256) + 1.47624 - 0.00223465 * cs1 + 1.36355 / cs1
    return y

def ng_nr_m2s(cs1):
    y = 28.1276*np.exp(-cs1/0.687178) + 1.38184 -0.00216714*cs1 + 0.450683/cs1
    return y

def ng_nr_p2s(cs1):
    y = 0.843921*np.exp(-cs1/106.891) + 1.03573 -1.0002e-05*cs1 + 2.1521/cs1
    return y

def er_mean(cs1):
    y = 0.413781*np.exp(-cs1/42.7873) + 1.53709 -0.000610923*cs1 + 1.82074/cs1
    return y

def er_m2s(cs1):
    y = 0.374125*np.exp(-cs1/33.5861) + 1.43559 -0.000755488*cs1 + 1.33334/cs1
    return y

def er_p2s(cs1):
    y = 0.483742*np.exp(-cs1/56.6718) + 1.60155 -0.000318184*cs1 + 2.3969/cs1
    return y

# 4 sigma checks
